In [4]:
%%html
<!-- definir quelques styles custom pour l'ensemble du notebook -->
<style>    
    @import url("css/custom_styles.css")
</style>

<center>
    <h1>
    Transformation Et Manipulation Des Données<br>
    </h1>
    MovieLens - Préparation des données pour système de recommendations de films.<br>
    <br>
    <b>Jean-Francois Gagnon</b><br>
    <b>Michèle de La Sablonnière</b><br>
    <br>
    420-A56<br>
    <br>
</center>

# Introduction
    
Nous avons choisi la base de données [MovieLens Small](https://tinyurl.com/bdhmcfht). Elle décrit les notations des utilisateurs de MovieLens; un service de recommendation de films. Elle contient plus de 100K notations représentant plus de 9700 films. Ces notations ont été établit par plus de 600 utilisateurs entre mars 1996 et septembre 2018. Elle contient également les métadonnées sur 19 genres. Les données sont structurées dans 4 fichiers distincts.

<br>**tags.csv**

Fichier contenant les tags (quelques mots générés par les utilisateurs) de chaque film. Ces données ne sont pas utilisées dans ce projet dans un but de réduire la portée.

<br>**links.csv**

Fichier contenant les metadonnées permettant de lier un film à d'autres source de données. Chaque ligne a ce format:

<div class="indentation">
<div class="fixblock">movieId, imdbId, tmdbId</div>

|Attribut|<center>Description</center>|
|:-|:---|
|movieId| Identifiant du film dans cette base de données.|
|imdbId| Identifiant du film dans [Internet Movie Database](http://www.imdb.com).|
|tmdbId| Identifiant du film dans [The Movie DB](https://www.themoviedb.org).|    
</div>

<br>**ratings.csv**

Ce fichier contient les notations. Chaque ligne a ce format:

<div class="indentation">
<div class="fixblock">userId, movieId, rating, timestamp</div>

|Attribut|<center>Description</center>|
|:-|:---|
|userId| Identifiant de l'utilisateur dans cette base de données.|
|movieId| Identifiant du film dans cette base de données.|
|rating| Nombre d'étoiles attribuées avec une granularité de $\frac{1}{2}$. |
|timestamp| Date à laquelle la note a été entré. Encodée dans le format [UTC](https://tinyurl.com/32e6a5pc).|
</div>

<br>**movies.csv**

Ce fichier contient les métadonnées de chaque film. Chaque ligne a ce format:

<div class="indentation">
<div class="fixblock">movieId, title, genres</div>

|Attribut|<center>Description</center>|
|:-|:---|
|userId| Identifiant de l'utilisateur dans cette base de données.|
|movieId| Identifiant du film dans cette base de données.|
|rating| Nombre d'étoiles attribuées avec une granularité de $\frac{1}{2}$. |
|timestamp| Date à laquelle la note a été entré. Encodée dans le format [UTC](https://tinyurl.com/32e6a5pc).|
</div>

<br>

L'objectif de ce projet est de préparer les données pour un éventuel système de recommentation de films.

In [ ]:
#
# imports utilitaires
#

%matplotlib inline

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns
import time

from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler, RobustScaler

from tqdm.notebook import tqdm

#
# imports faisant partie de mes propres module
#

import helpers as hlp
import helpers.Clustering as clstr
import helpers.dataset.MovieLens as mvl
import helpers.WebScraping as scrap
import helpers.jupyter as jup

# Prétraitement

<font class="answer">

Description ici?

In [ ]:
#
# parametres configurant nos traitemens
#
configs = hlp.get_configs("config_overrides.json")

#
# obtenir le dataset
#
mvl_dataset = mvl.load(configs.dataset)

## links.csv

<font class="answer">
    
links.csv n'est pas utilisé directement pour le clustering. Cependant, il le sera pour complémenter l'information des autres base de données. Il m'apparait donc imporant de faire un survol rapide.

In [ ]:
print("Links", mvl_dataset.links.shape)
print("Head")
display(mvl_dataset.links.head())
clstr.show_na(mvl_dataset.links)
clstr.show_types(mvl_dataset.links)

<font class="answer">
    
Il manque quelques liens sur [TMDB](https://www.themoviedb.org/) (ce qui explique le type float64). Les liens [IMDB](https://www.imdb.com/) seront par conséquent utilisés pour fin de web scrapping.

## movies.csv

<font class="answer">
    
Mettre description ici?

In [ ]:
print("Movies", mvl_dataset.movies.shape)
print("Head")
display(mvl_dataset.movies.head())
clstr.show_na(mvl_dataset.movies)
clstr.show_types(mvl_dataset.movies)

<font class="answer">

Tel que décris par MovieLens, *title* contient l'année de parution. Nous allons l'extraire afin d'obtenir une nouvelle variable. *title* sera adressé un pleu plus bas.

In [ ]:
def split_title_year(title, regex):
    x = regex.search(title)
    if x:
        title = x.group(1)
        year = x.group(2)
        year = int(year) if year else pd.NA
    else:
        year = pd.NA
    
    return pd.Series({"title": title, "year": year})

title_year_re = re.compile(configs.dataset.title_regex, flags=0)        
title_year = mvl_dataset.movies.title.apply(split_title_year, args=(title_year_re,))

#
# validation de l'extraction
#
print("Informations extraites", title_year.shape)
print("Head")
display(title_year.head())
clstr.show_na(title_year)
clstr.show_types(title_year)

<font class="answer">
   
On peut remarquer qu'il manque quelques années de parution. Il est probablement possible de les fixer en utilisant *links.imdbId* avec le web scrapping.

In [ ]:
def imdb_scap_year(response, index, imdbId):
    success = False
    if response.ok:
        bs = BeautifulSoup(response.text, "html.parser")

        imdb_info = bs.find("script", attrs={"type": "application/ld+json"})
        imdb_json = json.loads(imdb_info.text)
        date_published = scrap.get_nested_property(imdb_json, ["datePublished"])
        if not success and date_published:
            date_published = pd.to_datetime(date_published)
            year = date_published.year
            success = True

        imdb_info = bs.find("script", attrs={"type": "application/json"})
        imdb_json = json.loads(imdb_info.text)
        releaseYear = scrap.get_nested_property(imdb_json, 
                                                ["props", 
                                                 "pageProps", 
                                                 "mainColumnData", 
                                                 "releaseYear",
                                                 "year"])
        if not success and releaseYear:
            year = releaseYear
            success = True

    if success:
        return (index, year, response.url)
    else:
        return (index, None, response.url)

def imdb_scap_year_apply_results(final_results):
    for index, year, url in final_results:
        if year is None:
            print(url, title_year.title[index], "Failed")
        else:
            title_year.year[index] = year

#    
# corriger year via web scrapping avec imdb
#
year_na = title_year.year.isna()
year_link = mvl_dataset.links.imdbId[ year_na ]

if year_link.shape[0] > 0:
    with hlp.Profile() as year_profile:
        results = scrap.imdb_requests_parallel(year_link,
                                               configs.web_scraping,
                                               imdb_scap_year,
                                               executor=configs.executor)
        imdb_scap_year_apply_results(results)
    print(f"Web scraping year: {year_profile.round_duration(2)}s")

# validation du scapping
clstr.show_na(title_year);

In [ ]:
#
# mettre a jour movies
#
mvl_dataset.movies["year"] = title_year.year.astype(np.int64)

In [ ]:
#
# validation effet extraction de l'annee de title
#
imdb_ids = mvl_dataset.links.imdbId[mvl_dataset.links.movieId == mvl_dataset.movies.movieId]

title = mvl_dataset.movies[["movieId", "year"]].copy()
title["title"] = title_year.title.copy()
title["imdbId"] = imdb_ids.copy()

def title_agregate(dataframe):
    return pd.Series({"imdbIds": dataframe.imdbId.unique(),
                      "years": dataframe.year.unique(),
                      "counts": dataframe.movieId.nunique()})

groups = title.groupby("title").apply(title_agregate)
groups.sort_values(by="counts", ascending=False, inplace=True)

display(groups.head())

<font class="answer">
    
Une inspection manuelle de [Hamlet 1996](http://www.imdb.com/title/tt0116477) et [Hamlet 1948](http://www.imdb.com/title/tt0040416) via IMDB permet de voir que l'année juxtaposée au titre est un identifiant unique. Je dois donc garder *title* original.

In [ ]:
#
# validation doublons
#
print("Vérifier doublons")
print("Avant:", mvl_dataset.movies.shape)
mvl_dataset.movies.drop_duplicates(inplace=True)
print("Après:", mvl_dataset.movies.shape)

<font class="answer">
Aucun doublon

In [ ]:
#
# validation modalites genres
#
def show_unique_genres(sort_effectif=False):
    genres_count = {}

    def add(genre):
        if genre in genres_count:
            genres_count[genre] += 1
        else:
            genres_count[genre] = 1
    
    for genres_str in mvl_dataset.movies.genres:
        if genres_str is None:
            add(None)
        else:
            genre_array = genres_str.split(configs.dataset.genre_splitter)
            for genre in genre_array:
                add(genre)

    genres_df = pd.Series(data=genres_count.values(),
                          index=genres_count.keys(),
                          name="# films")

    if sort_effectif:
        #genres_df /= mvl_dataset.movies.shape[0] / 100
        genres_df.sort_values(ascending=False, inplace=True)
    else:
        genres_df.sort_index(inplace=True)

    print("Unique genres:", genres_df.shape[0])
    display(genres_df.to_frame())

show_unique_genres()

<font class="answer">

(no genres listed) semble être en fait des valeurs manquantes.

In [ ]:
#
# examiner (no genres listed)
#    
def show_no_genres():
    no_genres_crit = mvl_dataset.movies.genres.str.contains("(no genres listed)", regex=False)
    no_genres = mvl_dataset.movies[no_genres_crit]
    print("(no genres listed)", no_genres.shape[0])
    if no_genres.shape[0] > 0:
        display(no_genres.head())
    return no_genres.index
    
no_genres_index = show_no_genres()

<font class="answer">
Je vais utiliser le web scrapping pour tenter de remplacer (no genres listed).

In [ ]:
def imdb_scap_genres(response, index, imdbId):
    success = False
    if response.ok:
        bs = BeautifulSoup(response.text, "html.parser")

        imdb_info = bs.find("script", attrs={"type": "application/ld+json"})
        imdb_json = json.loads(imdb_info.text)
                
        genres = scrap.get_nested_property(imdb_json, ["genre"])
        if not success and genres:
            success = True
        else:
            error = json.dumps(imdb_json, indent=4)
    else:
        error = response.reason

    if success:
        return (index, genres, response.url, None)
    else:
        
        return (index, None, response.url, error)

def imdb_scap_genres_apply_results(final_results, genres_splitter):
    for index, genres, url, error in final_results:
        if genres is None:
            if False:
                print()
                print(error)
                print()

            print(url, mvl_dataset.movies.title[index], "Failed")
        else:
            mvl_dataset.movies.loc[index, "genres"] = genres_splitter.join(genres)


#
# corriger genres via web scrapping avec imdb
#
genres_imdbIds = mvl_dataset.links.imdbId[no_genres_index]

if genres_imdbIds.shape[0] > 0:
    with hlp.Profile() as genres_profile:
        if True:
            results = scrap.imdb_requests_parallel(genres_imdbIds, 
                                                   configs.web_scraping, 
                                                   imdb_scap_genres,
                                                   executor=configs.executor)
            imdb_scap_genres_apply_results(results, configs.dataset.genre_splitter)
        else:
            # les resultats ne sont pas appliques ici
            # ce code n'existe que pour mesurer le gain du multithreading
            scrap.imdb_requests(genres_imdbIds, 
                                configs.web_scraping, 
                                imdb_scap_genres)
    print(f"Web scraping genres: {genres_profile.round_duration(2)}s")
        
show_unique_genres(sort_effectif=True)
show_no_genres();

<font class="answer">
    
Par souci de réduire la quantité de modalités, toutes celles en dessous de 4 individus seront enlevées. Noter que Music et Musical sont synonyme. Finalement, IMAX n'est pas un genre mais bien un format de diffusion.

In [ ]:
def clean_genres(genres_str, splitter, to_remove):
    if genres_str is None or not isinstance(genres_str, str):
        return genres_str
    
    genres_list = genres_str.split(splitter)
    genres_set = set(genres_list)
    genres_set -= to_remove
    
    # bien que non portable, par souci d'efficacite
    # handler Music et Musical
    if "Music" in genres_set:
        genres_set.remove("Music")
        genres_set.add("Musical")

    genres_str = splitter.join(genres_set)
    if len(genres_str) == 0:
        genres_str = None

    return genres_str


#
# nettoyer les modalites de genres
#
to_remove = ["IMAX", "Short", "Biography", "Family", "History"]
to_remove = set(to_remove)
cleaned_genres = mvl_dataset.movies.genres.apply(clean_genres, 
                                                 splitter=configs.dataset.genre_splitter,
                                                 to_remove=to_remove)
mvl_dataset.movies["genres"] = cleaned_genres

print("Genres nettoyés")
genres_na = clstr.show_na(mvl_dataset.movies)
print(mvl_dataset.movies.shape)

<font class="answer">
Certains films avaient 1 seul genre. Après le retrait précédant, 

In [ ]:
mvl_dataset.movies.drop(index=genres_na, axis=0, inplace=True, errors="ignore")

# validation retrain NA de genres
show_unique_genres(sort_effectif=True)
clstr.show_na(mvl_dataset.movies);

<font class="answer">

Afin de simplifier la partie exploration, je vais ajouter *imdbId* à *movies.csv* afin d'avoir l'information dans une seule base de données.

In [ ]:
#
# ajout imdbId a movies
#
mvl_dataset.movies = pd.merge(mvl_dataset.movies, 
                              mvl_dataset.links[["movieId", "imdbId"]],
                              how="left", 
                              on="movieId")

# pour faciliter la visualization, mettre immediatement imdbId
mvl_dataset.movies.insert(1, 'imdbId', mvl_dataset.movies.pop('imdbId'))

# validation
print("Validation ajout imdbId")
display(mvl_dataset.movies.head())

## ratings.csv

<font class="answer">
    
Mettre description ici?

In [ ]:
print("Ratings", mvl_dataset.ratings.shape)
print("Head")
display(mvl_dataset.ratings.head())
clstr.show_na(mvl_dataset.ratings)
clstr.show_types(mvl_dataset.ratings)

<font class="answer">
  
*rating*, bien que montré en nombre flottant, est en fait un système d'étoiles ou la mesure de base est$\frac{1}{2}$. Je garde en quantitatif pour le moment car il semble avantageux pour faire un lien avec les données dans *movies.csv*. L'exploration pourra décider s'il est pertinant de passer en catégoriel si on traite les *ratings.csv* à part (système de recommendation collaboratif).
    
*timestamp* est en fait une date dans le format UTC. Ceci semble suggérer qu'un utilisateur peut avoir voté plus d'une fois pour le même film ou utiliser un système pour automatisé pour biaser les résultats.

In [ ]:
#
# verification nombre de votes par couple (utilisateur, film)
#
votes = mvl_dataset.ratings.groupby(by=["userId", "movieId"]).size().max()
print("Nombre de votes par (userId, movieId):", votes)

<font class="answer">
Un seul vote par utilisateur par film.

In [ ]:
#
# verification frequence des votes par utilisateur
#
def dt_sec(data):
    return data.max() - data.min()

def dt_expanded(data):
    utc_min = datetime.utcfromtimestamp(data.min())
    utc_max = datetime.utcfromtimestamp(data.max())
    dt_full = relativedelta(utc_max, utc_min)
    
    full = []
    if dt_full.years > 0:
        full.append(f"{dt_full.years}y")
    elif dt_full.months > 0:
        full.append(f"{dt_full.months}m")
    elif dt_full.days > 0:
        full.append(f"{dt_full.days}d")
    elif dt_full.hours > 0:
        full.append(f"{dt_full.hours}h")
    else:
        if dt_full.minutes > 0:
            full.append(f"{dt_full.minutes}m")

        if dt_full.seconds > 0:
            full.append(f"{dt_full.seconds}s")
    
    return " ".join(full)

def rating_freq(data):
    num = data.shape[0]
    freq = num / dt_sec(data)
    # mHz pour mieux voir ecarts
    return freq * 1000

user_ratings = mvl_dataset.ratings[["userId", "rating", "timestamp"]] \
                          .groupby(by="userId") \
                          .agg(rating_count=("rating", "count"), 
                               dt_sec=("timestamp", dt_sec),
                               dt_expanded=("timestamp", dt_expanded),
                               rating_freq=("timestamp", rating_freq))

rating_freq_stats = user_ratings.rating_freq.describe().to_frame()
rating_cout_stats = user_ratings.rating_count.describe().to_frame()
fastest_users = user_ratings.nlargest(5, "rating_freq")
slowest_users = user_ratings.nsmallest(5, "rating_freq")

html = jup.horizontify(jup.caption_df(fastest_users.style, "User les plus rapide", caption_bold=False), 
                       jup.caption_df(slowest_users.style, "User les moins rapide", caption_bold=False))
jup.display_html(html)

print()
print("Stats générale")
html = jup.horizontify(rating_freq_stats.style, rating_cout_stats.style)
jup.display_html(html)

In [ ]:
#
# visualiser rating_count vs rating_freq
#
def rating_count_vs_freq(user_ratings, critOutliers=None, figsize=(7, 3)):
    plt.figure(figsize=figsize)
    
    if critOutliers is None:
        plt.scatter(user_ratings.rating_count, user_ratings.rating_freq, s=5)
    else:
        outliers = user_ratings[critOutliers]
        plt.scatter(outliers.rating_count, outliers.rating_freq, s=5, label="Outliers")
        
        inliers = user_ratings[~critOutliers]
        plt.scatter(inliers.rating_count, inliers.rating_freq, s=5, label="Inliers")
        
        plt.legend()
    
    plt.grid(True)
    plt.xlabel("Rating Count")
    plt.ylabel("Rating Freq (mHz)")
    plt.show()
    
rating_count_vs_freq(user_ratings)

<font class="answer">
Les utilisateurs ont votés relativement souvent (20 votes ou plus) mais la distribution est très allongée et fortement concentrée dans les valeurs faible. Le graphe ci-haut suggère que les votes semblent avoir étés fait de façon très rapide. Pour illustrer, le user *163* aurait voté 23 fois en 38 secondes (~2 secondes par vote). mais le user *172* aurait pris 1 an pour ses 26 votes. Faute d'information supplémentaire, je suspecte un système automatisé et enlève les utilisateurs ayant une fréquence de vote jugée trop élevée.<br><br>
    
En absolue:
$$
\begin{align}
threshold &= Q_3 + 1.5 * IQR\\
          &= 62.19 + 1.5 * (62.19 - 0.04)\\
          &= 155.36\\
\end{align}
$$
  
Une approche basée sur le clustering, plus particulièrement DBSCAN, trouvera les éléments isolés dans le graphe.

In [ ]:
coords = RobustScaler().fit_transform(user_ratings[["rating_count", "rating_freq"]])
clstr.dbscan_eps_analysis(coords, figsize=(6, 3.5))

eps_, min_samples_ = clstr.dbscan_parameters_analysis(coords, 
                                                      np.arange(0.25, 0.45, 0.01),
                                                      range(2, 20))
dbscan_ = clstr.dbscan_init(coords, eps_, min_samples_)
clstr.show_clusters(coords, 
                    user_ratings.index, 
                    dbscan_.labels_, 
                    figsize=(7, 3.5),
                    text_alpha=0, 
                    marker_size=5)

In [ ]:
#
# detecter les user juges outliers
#
critA = dbscan_.labels_ == -1

# dbscan marque certaines basses frequences 
# s'assurer de les garder
critB = user_ratings.rating_freq > 1

# IQR threshold
critC = user_ratings.rating_freq >= 155.36

# combiner tous les criteres d'outliers
critOutliers = critA & critB | critC

print("Nombre de user outliers", np.count_nonzero(critOutliers))
rating_count_vs_freq(user_ratings, critOutliers)

In [ ]:
#
# enlever les user juges outliers
#
user_to_remove = user_ratings[critOutliers].index.to_list()
user_to_remove = mvl_dataset.ratings.userId.isin(user_to_remove)
user_to_remove = mvl_dataset.ratings[user_to_remove]

print("ratings avant retrait users:", mvl_dataset.ratings.shape[0])
mvl_dataset.ratings.drop(user_to_remove.index, inplace=True, errors="ignore")
print("ratings après retrait users:", mvl_dataset.ratings.shape[0])

<font class="answer">
Pour movies.csv, il apparait judicieux d'extraire quelques statistiques sur *rating*

In [ ]:
def etendue(ratings):
    return ratings.max() - ratings.min()

def iqr(ratings):
    return ratings.quantile(0.75) - ratings.quantile(0.25)

def mode(ratings):
    # prendre le mode le plus grand
    return ratings.mode().iloc[-1]

def median_keep_category(ratings):
    # garder les modalites de ratings si nombre de valeurs est impaire
    return ratings.quantile(interpolation="nearest")

def join_name(multi_index_name):
    if "" in multi_index_name:
        return multi_index_name[0]
    else:
        return "_".join(multi_index_name)


#
# creer des variables pour movies
#
ratings_stats = mvl_dataset.ratings[["movieId", "rating"]] \
                           .groupby(by="movieId") \
                           .agg(["count", ("mode", mode), "mean", ("median", median_keep_category)])
ratings_stats.columns = [join_name(a) for a in ratings_stats.columns.to_flat_index()]
ratings_stats.reset_index(inplace=True)
print("Rating stats", ratings_stats.shape)
print("Head")
display(ratings_stats.head().round(2))

In [ ]:
#
# detailler les valeurs manquantes
#
clstr.show_na(ratings_stats);

In [ ]:
#
# valider que ratings_stats contient bien les meme films que movies.csv
#
movies_ratings = pd.merge(mvl_dataset.movies, ratings_stats, how="left", on="movieId")
movies_ratings_na = clstr.show_na(movies_ratings)

<font class="answer">
    
Tous les films n'ont pas reçu de votes. Je pourrais utiliser le web scraping pour remplir rating_mean mais l'echelle sur IMDB n'est pas la même que MovieLens. De plus les autres statistiques ne pourraient pas être extraite. Comme il n'y a que très peux d'éléments manquants je vais les ignorer.

In [ ]:
#
# enlever valeurs manquantes et valider shape resultante
#
print("movies_ratings avant retrait:", movies_ratings.shape)
movies_ratings.drop(index=movies_ratings_na, axis=0, inplace=True, errors="ignore")
print("movies_ratings après retrait:", movies_ratings.shape)
print()
#
# valider les types (drop() peut changer les choses du aux NA)
#
clstr.show_types(movies_ratings)

In [ ]:
#
# rating_count est en fait un entier, changer son type
#
movies_ratings.rating_count = movies_ratings.rating_count.astype(np.int64)

In [ ]:
#
# sauvegarde des traitements sur movies_ratings
#
csv_path = mvl_dataset.movies_path.split(".")[0]
csv_path = "".join([csv_path, "_pretraitement", ".csv"])

print("Sauvegarde", csv_path)
movies_ratings.to_csv(csv_path, index=False)

## Collaborative filtering<br>
<font class="answer">
Les systèmes de votes ont souvent 2 composantes: content base et collaborative filtering. L'idée étant d'utiliser l'historique d'un utilisateur (collaborative filtering) et à défaut de l'avoir, utiliser seulement le contenu des fims. Notre base de donnée est plutot simple, nous auront donc seulement rating pour regrouper les users.

In [ ]:
#
# s'assurer de garder uniquement les uers qui ont les movieId present dans movies_ratings
#
movieIds = movies_ratings.movieId.unique()
ratings = mvl_dataset.ratings[ mvl_dataset.ratings.movieId.isin(movieIds)  ]
user_ratings = ratings.pivot(index="userId", columns="movieId", values="rating")

display(user_ratings)

<font class="answer">
Données très éparse. Ceci est attendu puisque le système veux faire des suggestion.

In [ ]:
#
# sauvegarde des traitements sur user_ratings
#
csv_path = mvl_dataset.ratings_path.split(".")[0]
csv_path = "".join([csv_path, "_pretraitement", ".csv"])

print("Sauvegarde", csv_path)
user_ratings.to_csv(csv_path, index=False)